In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from utils_np import *
from visualization import *

## Numerical methods

We define the midpoint method and explicit Euler for the angular velocity $\omega$ and Lie Euler for the orientation $R$. We use a simple fixed point iteration to solve the implicit midpoint method.

In [ ]:
def fixed_point_iter(x0,dt,f_method):
    w0,_ = x0
    w1 = w0
    x1 = (w1,None)
    while np.linalg.norm(w1 - (w0 + dt*f_method(x0,x1))) > 1e-15:
        w1 = w0 + dt*f_method(x0,x1)
        x1 = (w1,None)
    return w1

def midpoint(x0,dt,f):
    
    def f_method(x0,x1):
        w0,R0 = x0
        w1,_ = x1
        return f(.5*(w0+w1),R0)
    
    return fixed_point_iter(x0,dt,f_method)

def explicit_euler(x0,dt,f):
    w0,R0 = x0
    return w0 + dt*f(w0,R0)

def implicit_euler(x0,dt,f):
    
    def f_method(x0,x1):
        w0,R0 = x0
        w1,_ = x1
        return f(w1,R0)
    
    return fixed_point_iter(x0,dt,f_method)


def lie_euler(x0,dt,f):
    w0,R0 = x0
    return R0@rodrigues(dt*f(w0))



In [ ]:
def integrate_rigid_body(w0,R0,dt,N,f_w,f_R,phi_w,phi_R):
    """ 
    w0:     initial value angular velocity (R^3)
    R0:     initial value orientation (SO(3))
    dt:     step size in time
    N:      number of time steps
    f_w:    vector field for anngular velocity
    f_R:    vector field for orientation
    phi_w:  integrator for angular velocity
    phi_R:  integrator for orientation
    """
    Rs = [R0]
    ws = [w0]

    for n in range(N):
        R0 = Rs[-1]
        w0 = ws[-1]
        x0 = (w0,R0)
        ws.append(phi_w(x0,dt,f=f_w))
        Rs.append(phi_R(x0,dt,f=f_R))

    Rs = np.stack(Rs)
    ws = np.stack(ws)

    return ws,Rs



## Solving IVP for a uncontrolled rigid body

We first set initial values and the moments of inertia, and define the time derivative of the angular velocity and orientation. Additionally, we define the Hamiltonian as a function of $\omega$.

In [ ]:
#Initial values
R0 = get_orth(3,seed=10)
w0 = np.array([-0.1,0.1,0.3])

#Moments of inertia and inverse
diag_I = np.array([1,0.8,1.7])
I = np.diag(diag_I)
I_inv = np.diag(1/diag_I)

#Defining the time derivatives
w_dot = lambda w,R : I_inv@hat(I@w)@w 
R_dot = lambda w : hat(w)

#Hamiltonian, which is just the rotational kinetic energy (recall p = I@w and H(p) = .5*p.T@I@p)
hamiltonian = lambda w : 0.5*I@w.T@I_inv@I@w

In [ ]:
#Step size and number of steps
dt = 0.1
N = 1000

#Choosing vector fields and integrators
f_w =       w_dot
f_R =       R_dot
phi_w =     explicit_euler
phi_R =     lie_euler

ws,Rs = integrate_rigid_body(w0,R0,dt,N,f_w,f_R,phi_w,phi_R)
plot_energy_orth(hamiltonian,dt,ws,Rs)
plot_trajectory_S2(R0,Rs,i=1)

## Applying ES & DI control for a rigid body

Setting initial values and the target orientation ```Rref``` as well as gains ```Kp,Kd``` for the controls.

In [ ]:
#Initial values
R0 = get_orth(3,seed=10)
w0 = np.array([-0.1,0.1,0.3])
Rref = get_orth(3,seed=0)


#Controller gains and desired orientation
Kp = np.diag([1]*3)
Kd = Kp

#Defining the energy shaping and damping injection controls
u_es = lambda R: -0.5*hat_inv(Kp@Rref.T@R - R.T@Rref@Kp.T)
u_di = lambda w: -Kd@w

#Time derivatives with controls
w_dot_es_di = lambda w,R : I_inv@(hat(I@w)@w + u_es(R) + u_di(w) )
R_dot = lambda w : hat(w) 

#Injected control energy H_ref (for plotting)
hamiltonian_ref = lambda R : .5*np.trace(Kp@(np.eye(3) - Rref.T@R))
yTu_midpoint = lambda R,w0,w1 :  .5*(w0+w1).T@(u_es(R) + u_di(.5*(w0+w1)))
yTu_euler = lambda R,w0,w1 :  w0.T@(u_es(R) + u_di(w0))




## Integrating controlled rigid body with explicit Euler

In [ ]:
#Step size and number of steps
dt = 0.2
N = 100

#Choosing vector fields and integrators
f_w =       w_dot_es_di  
f_R =       R_dot
phi_w =     explicit_euler
phi_R =     lie_euler
yTu =       yTu_euler


ws,Rs = integrate_rigid_body(w0,R0,dt,N,f_w,f_R,phi_w,phi_R)
plot_discrete_energy_balance(hamiltonian,yTu_euler,dt,ws,Rs)
plot_energy_orth(hamiltonian,dt,ws,Rs,H_ref = hamiltonian_ref)
plot_trajectory_S2(R0,Rs,Rref=Rref,i=2)

## Integrating controlled rigid body with implicit Euler

In [ ]:
#Step size and number of steps
dt = 0.2
N = 100

f_w =       w_dot_es_di
f_R =       R_dot
phi_w =     midpoint
phi_R =     lie_euler
yTu =       yTu_midpoint


ws,Rs = integrate_rigid_body(w0,R0,dt,N,f_w,f_R,phi_w,phi_R)
plot_discrete_energy_balance(hamiltonian,yTu,dt,ws,Rs)
plot_energy_orth(hamiltonian,dt,ws,Rs,H_ref = hamiltonian_ref)
plot_trajectory_S2(R0,Rs,Rref=Rref,i=2)